In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('../NY_sample.csv')
data = data.drop(['store_and_fwd_flag', 'congestion_surcharge', 'payment_type'], axis=1)
# Convert to datetime and extract features
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])
data['pickup_hour'] = data['tpep_pickup_datetime'].dt.hour
data['pickup_day_of_week'] = data['tpep_pickup_datetime'].dt.dayofweek
data['pickup_month'] = data['tpep_pickup_datetime'].dt.month
data['dropoff_hour'] = data['tpep_dropoff_datetime'].dt.hour
data['dropoff_day_of_week'] = data['tpep_dropoff_datetime'].dt.dayofweek
data['dropoff_month'] = data['tpep_dropoff_datetime'].dt.month

C:\Users\27143\AppData\Local\Temp\ipykernel_3216\1702762657.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../NY_sample.csv')


In [3]:
print(data['pickup_day_of_week'])

0         0
1         0
2         0
3         0
4         0
         ..
549755    1
549756    1
549757    1
549758    1
549759    1
Name: pickup_day_of_week, Length: 549760, dtype: int64


In [4]:
# Filter the dataset for similar distance trips
similar_distance_data = data

# Create a new column for pickup hour ranges
def map_hour_to_range(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

# Create the 'pickup_hour_range' column
data['pickup_hour_range'] = data['pickup_hour'].apply(map_hour_to_range)

# Calculate the tip percentage
data['tip_percentage'] = (data['tip_amount'] / data['fare_amount']) * 100

# Define the distance bins and time ranges
distance_bins = np.arange(0, data['trip_distance'].max() + 0.5, 0.5)
time_ranges = ['morning', 'afternoon', 'evening', 'night']

# Function to calculate the tip percentage distribution for a given time range
def calculate_tip_percentage_distribution(time_range_data):
    tip_percentage_counts = time_range_data.groupby(pd.cut(time_range_data['trip_distance'], distance_bins))[
        'tip_percentage'].value_counts(normalize=True).reset_index(name='percentage')
    return tip_percentage_counts

# Calculate the tip percentage distribution for each time range
time_range_data = {}
for time_range in time_ranges:
    time_range_data[time_range] = calculate_tip_percentage_distribution(data[data['pickup_hour_range'] == time_range])

# Plot the tip percentage distribution for each time range
plt.figure(figsize=(12, 8))
for time_range in time_ranges:
    sns.lineplot(data=time_range_data[time_range], x='tip_percentage', y='percentage', label=time_range)

plt.xlabel('Tip Percentage')
plt.ylabel('Percentage of People Paying Tip')
plt.title('Tip Percentage Distribution by Time Range')
plt.legend()
plt.show()




ValueError: operands could not be broadcast together with shape (440773,) (122,)

In [ ]:
similar_distance_data = data[(data['trip_distance'] >= 1) & (data['trip_distance'] <= 1.5)]

# Create a new column for pickup hour ranges
def map_hour_to_range(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 19:
        return 'afternoon'
    elif 19 <= hour < 21:
        return 'evening'
    else:
        return 'night'

def map_day_to_range(day):
    if 0 <= day < 5:
        return 'weekday'
    else:
        return 'weekend'

similar_distance_data['pickup_hour_range'] = similar_distance_data['pickup_hour'].apply(map_hour_to_range)
similar_distance_data['pickup_day_range'] = similar_distance_data['pickup_day_of_week'].apply(map_day_to_range)
similar_distance_data['percent_not_tipped'] = 1 - (similar_distance_data['total_amount'] - similar_distance_data['tip_amount']) / similar_distance_data['total_amount']


In [ ]:
print(similar_distance_data['pickup_day_range'])

In [ ]:

# Filter the dataset for the required pickup hour ranges
count_1 = similar_distance_data[similar_distance_data['passenger_count'] == 1]
count_2 = similar_distance_data[similar_distance_data['passenger_count'] == 2]
count_3 = similar_distance_data[similar_distance_data['passenger_count'] == 3]
count_4 = similar_distance_data[similar_distance_data['passenger_count'] == 4]
count_5 = similar_distance_data[similar_distance_data['passenger_count'] == 5]
count_6 = similar_distance_data[similar_distance_data['passenger_count'] == 6]

# Define the bins for the percent_not_tipped values
bins = np.arange(0, 1, 0.01)

# Calculate the percentage of passengers falling in each bin for each pickup hour range
c1_pct = np.histogram(count_1['percent_not_tipped'], bins=bins)[0] / len(count_1) * 100
c2_pct = np.histogram(count_1['percent_not_tipped'], bins=bins)[0] / len(count_2) * 100
c3_pct = np.histogram(count_1['percent_not_tipped'], bins=bins)[0] / len(count_3) * 100
c4_pct = np.histogram(count_1['percent_not_tipped'], bins=bins)[0] / len(count_4) * 100
c5_pct = np.histogram(count_1['percent_not_tipped'], bins=bins)[0] / len(count_5) * 100
c6_pct = np.histogram(count_1['percent_not_tipped'], bins=bins)[0] / len(count_6) * 100

# Define the x-axis labels for the bins
bin_labels = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins)-1)]

# Define the colors for each line
colors = ['red', 'blue', 'green', 'yellow','purple','white']

# Plot the percentage of passengers in each bin for each pickup hour range
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(bin_labels, c1_pct, label='1', color= colors[0])
ax.plot(bin_labels, c2_pct, label='2', color= colors[1])
ax.plot(bin_labels, 1, label='3', color= colors[2])
ax.plot(bin_labels, c4_pct, label='4', color= colors[3])
ax.plot(bin_labels, c5_pct, label='5', color= colors[4])
ax.plot(bin_labels, c6_pct, label='6', color= colors[5])


ax.set_xlabel('Percent Not Tipped')
ax.set_ylabel('Percentage of Passengers')
ax.set_title('Percentage of Passengers by Percent Tipped and Pickup Hour Range')
ax.legend()
plt.figure(figsize=(240,120))
plt.show()



In [ ]:
print(np.histogram(count_1['percent_not_tipped'], bins=bins)[0]/ len(count_1) * 100)

In [ ]:
weekday_data = similar_distance_data[similar_distance_data['pickup_day_range'] == 'weekday']
weekend_data = similar_distance_data[similar_distance_data['pickup_day_range'] == 'weekend']
bins = np.arange(-0.1, 0.4, 0.01)

weekday_pct = np.histogram(weekday_data['percent_not_tipped'], bins=bins)[0] / len(weekday_data) * 100 -np.histogram(weekend_data['percent_not_tipped'], bins=bins)[0] / len(weekend_data) * 100
# weekend_pct =

# Define the x-axis labels for the bins
bin_labels = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins)-1)]

colors = ['red', 'blue', 'green', 'yellow']

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(bin_labels, weekday_pct, label='weekday', color=colors[0])
# ax.plot(bin_labels, weekend_pct, label='weekend', color=colors[1])

ax.set_xlabel('Percent Not Tipped')
ax.set_ylabel('Percentage of Passengers')
ax.set_title('Percentage of Passengers by Percent Tipped and Pickup Hour Range')
ax.legend()
plt.figure(figsize=(240,120))
plt.show()

In [ ]:
# num_zero_tips = ((similar_distance_data['tip_amount'] == 0) & (similar_distance_data['pickup_day_range'] == 'weekend')).sum()
num_zero_tips = ((similar_distance_data['tip_amount'] == 0) & (similar_distance_data['pickup_day_range'] == 'weekday')).sum()
# Calculate the total number of tip amounts
# print(num_zero_tips)
total_tips = len(similar_distance_data['pickup_day_range'] == 'weekday')
# total_tips = len(similar_distance_data['pickup_day_range'] == 'weekend')
# Calculate the ratio of zero tips to total tips
zero_tip_ratio = num_zero_tips / total_tips

# Calculate the ratio of zero tips to non-zero tips
nonzero_tip_ratio = (total_tips - num_zero_tips) / total_tips

print("Ratio of zero tips to total tips: ", zero_tip_ratio)
print("Ratio of non-zero tips to total tips: ", nonzero_tip_ratio)

In [ ]:

# Filter the dataset for the required pickup hour ranges
night_data = similar_distance_data[similar_distance_data['pickup_hour_range'] == 'night']
evening_data = similar_distance_data[similar_distance_data['pickup_hour_range'] == 'evening']
morning_data = similar_distance_data[similar_distance_data['pickup_hour_range'] == 'morning']
afternoon_data = similar_distance_data[similar_distance_data['pickup_hour_range'] == 'afternoon']

# Define the bins for the percent_not_tipped values
bins = np.arange(0, 0.4, 0.01)

# Calculate the percentage of passengers falling in each bin for each pickup hour range
night_pct = np.histogram(night_data['percent_not_tipped'], bins=bins)[0] / len(night_data) * 100
evening_pct = np.histogram(evening_data['percent_not_tipped'], bins=bins)[0] / len(evening_data) * 100
morning_pct = np.histogram(morning_data['percent_not_tipped'], bins=bins)[0] / len(morning_data) * 100
afternoon_pct = np.histogram(afternoon_data['percent_not_tipped'], bins=bins)[0] / len(afternoon_data) * 100

# Define the x-axis labels for the bins
bin_labels = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins)-1)]

# Define the colors for each line
colors = ['red', 'blue', 'green', 'yellow']

# Plot the percentage of passengers in each bin for each pickup hour range
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(bin_labels, night_pct, label='Night', color=colors[0])
ax.plot(bin_labels, evening_pct, label='Evening', color=colors[1])
ax.plot(bin_labels, morning_pct, label='Morning', color=colors[2])
ax.plot(bin_labels, afternoon_pct, label='Afternoon', color=colors[3])
ax.set_xlabel('Percent Not Tipped')
ax.set_ylabel('Percentage of Passengers')
ax.set_title('Percentage of Passengers by Percent Tipped and Pickup Hour Range')
ax.legend()
plt.figure(figsize=(240,120))
plt.show()



In [ ]:
# Calculate the number of zero tip amounts
num_zero_tips = (data['tip_amount'] == 0).sum()

# Calculate the total number of tip amounts
total_tips = len(data)

# Calculate the ratio of zero tips to total tips
zero_tip_ratio = num_zero_tips / total_tips

# Calculate the ratio of zero tips to non-zero tips
nonzero_tip_ratio = (total_tips - num_zero_tips) / total_tips

print("Ratio of zero tips to total tips: ", zero_tip_ratio)
print("Ratio of non-zero tips to total tips: ", nonzero_tip_ratio)
